Simulation results (calibrate coefficients and run TES) of two SW-TES like methods  
The evaluation results are plotted in Matlab (density scatter)

In [1]:
import numpy as np
import pickle
from tqdm import tqdm, trange
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error
from matplotlib import pyplot as plt
from scipy.io import savemat
from lib.tes import *

plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 12

tes_calculator = TES_calculator_vec(sensor='ecostress', n_bands=3)
swdtes_calculator = SWDTES_calculator_vec()
lamda_c = swdtes_calculator.lamda_c

# SW-TES (Zheng et al., 2019, 2022)  
## Calibrate coefficients

In [2]:
## load simulation dataset
Lt_simu = np.load('data/simulation_dataset/Lt_simu.npy')[...,[1,3,4]]
Lg_simu = np.load('data/simulation_dataset/Lg_simu.npy')[...,[1,3,4]]
num_atm, num_vza, num_lst, num_emi, num_channel  = Lt_simu.shape

# convert radiance to brightness temperature
Tg_simu = Inverse_Planck_law(Lg_simu, lamda_c)
Tt_simu = Inverse_Planck_law(Lt_simu, lamda_c)

idx = pickle.load(open('data/simulation_dataset/idx.pkl', 'rb'))
idx_train, idx_val, idx_test = idx['train'], idx['val'], idx['test']

Tt_train, Tt_test = Tt_simu[idx_train].reshape(-1, 3), Tt_simu[idx_test].reshape(-1, 3)
Tg_train, Tg_test = Tg_simu[idx_train].reshape(-1, 3), Tg_simu[idx_test].reshape(-1, 3)

In [3]:
# find the best combination of channels and calibrate the coefficients
def regress_coefs(x, y, num_channel=3):
    model = LinearRegression()
    rmse_Tg = np.ones((num_channel, num_channel)) * np.Inf
    slopes = np.zeros((num_channel, num_channel, 3))
    intercepts = np.zeros((num_channel, num_channel))
    for i in trange(num_channel):
        yi = y[:, i]
        xi = x[:, i]
        for j in range(num_channel):
            if i==j:
                continue
            xj = x[:, j]
            dx_ij = xi - xj
            dx_ij_2 = dx_ij**2
            inputs_ij = np.concatenate((xi[:,None], dx_ij[:,None], dx_ij_2[:,None]), axis=1)

            model.fit(inputs_ij, yi)
            yi_prd = model.predict(inputs_ij)
            rmse_Tg[i, j] = root_mean_squared_error(yi, yi_prd)

            slopes[i, j, :] = model.coef_
            intercepts[i, j] = model.intercept_
    return rmse_Tg, slopes, intercepts

rmse_Tg, slopes, intercepts = regress_coefs(Tt_train, Tg_train, num_channel=3)
idx_i = np.arange(3)
idx_j = np.argmin(rmse_Tg, axis=1)

slopes_opt = slopes[idx_i, idx_j]
intercepts_opt = intercepts[idx_i, idx_j]
coefs_opt = np.concatenate((intercepts_opt[:,None], slopes_opt), axis=1)
# np.save('result/sw_tes/coefs_sw_tes.npy', coefs_opt.astype(np.float32))

100%|██████████| 3/3 [00:05<00:00,  1.75s/it]


## Estimate Lg and Ld on the test set

In [31]:
# validate on the test set
# note: the inputs of SW-TES and SWDTES are not added with noise, but the performance is similar
coefs = np.load('result/sw_tes/coefs_sw_tes.npy')
Tt_b2_test, Tt_b4_test, Tt_b5_test = Tt_test[:,0], Tt_test[:,1], Tt_test[:,2]
Tg_b2_test, Tg_b4_test, Tg_b5_test = Tg_test[:,0], Tg_test[:,1], Tg_test[:,2]

d_Tt_b2_b5 = Tt_b2_test - Tt_b5_test
d_Tt_b4_b5 = Tt_b4_test - Tt_b5_test
d_Tt_b5_b4 = Tt_b5_test - Tt_b4_test

X_b2_test = np.concatenate((np.ones_like(Tt_b2_test[:,None]), Tt_b2_test[:,None], d_Tt_b2_b5[:,None], d_Tt_b2_b5[:,None]**2), axis=1)
X_b4_test = np.concatenate((np.ones_like(Tt_b4_test[:,None]), Tt_b4_test[:,None], d_Tt_b4_b5[:,None], d_Tt_b4_b5[:,None]**2), axis=1)
X_b5_test = np.concatenate((np.ones_like(Tt_b4_test[:,None]), Tt_b5_test[:,None], d_Tt_b5_b4[:,None], d_Tt_b5_b4[:,None]**2), axis=1)

Tg_test_prd = np.zeros((len(Tt_b2_test), 3))
Tg_test_prd[:,0] = X_b2_test @ coefs[0]
Tg_test_prd[:,1] = X_b4_test @ coefs[1]
Tg_test_prd[:,2] = X_b5_test @ coefs[2]
rmse_Tg = root_mean_squared_error(Tg_test_prd, Tg_test, multioutput='raw_values')
print(f'rmse_Tg: {rmse_Tg}')

Lg_test_prd = Planck_law(Tg_test_prd, lamda_c)
Lg_test_true = Planck_law(Tg_test, lamda_c)
rmse_Lg = root_mean_squared_error(Lg_test_prd, Lg_test_true, multioutput='raw_values')
print(f'rmse_Lg: {rmse_Lg}')

# np.save('result/sw_tes/Lg_sw_tes.npy', Lg_test_prd.astype(np.float32))

rmse_Tg: [1.53492566 1.25432876 2.14442206]
rmse_Lg: [0.29093686 0.21235156 0.28033942]


In [32]:
Ld_modtran = np.load('result/sw_tes/Ld_modtran.npy')
Ld_LUT = np.load('result/sw_tes/Ld_LUT.npy')
Ld_true = np.load('data/simulation_dataset/Ld.npy')[..., [1,3,4]]
lat_month_profile = np.load('data/profile_info/lat_month_clear.npy')
lat, month = lat_month_profile[:,0], lat_month_profile[:,1]

'''
# T:0, MLS:1, MLW:2, SAS:3, SAW:4
Ld_LUT = np.array([[4, 4, 4, 2, 2, 4], # 80
            [4, 4, 2, 2, 2, 4], # 70
            [2, 2, 2, 3, 3, 2], # 60
            [2, 2, 3, 3, 3, 3], # 50
            [3, 3, 3, 1, 1, 3], # 40
            [1, 1, 1, 0, 0, 1], # 30
            [0, 0, 0, 0, 0, 0], # 20
            [0, 0, 0, 0, 0, 0], # 10
            [0, 0, 0, 0, 0, 0], # 0
            [0, 0, 0, 0, 0, 0], # -10
            [0, 0, 0, 1, 1, 0], # -20
            [1, 1, 1, 1, 1, 1], # -30
            [3, 3, 3, 3, 3, 3], # -40
            [3, 3, 3, 2, 2, 3], # -50
            [2, 2, 2, 2, 2, 2], # -60
            [2, 2, 2, 2, 2, 2], # -70
            [2, 2, 2, 4, 2, 2], # -80
            ])
'''

row_in_lut = np.round((80-lat)/10).astype(int)
row_in_lut[row_in_lut==-1] = 0
row_in_lut[row_in_lut==17] = 16
col_in_lut = ((month-1) // 2).astype(int)
idx = Ld_LUT[row_in_lut, col_in_lut]
Ld_lut = Ld_modtran[idx][:,[1,3,4]]
rmse_Ld = root_mean_squared_error(Ld_lut, Ld_true, multioutput='raw_values')
print(f'rmse_Ld: {rmse_Ld}')

# np.save('result/sw_tes/Ld_sw_tes.npy', Ld_lut.astype(np.float32))
# savemat('matlab_plot/data/rad_sw_tes.mat', {'Lg_true': Lg_test_true.astype(np.float32), 'Lg_prd': Lg_test_prd.astype(np.float32), 'Ld_true': Ld_true.astype(np.float32), 'Ld_prd': Ld_lut.astype(np.float32)})

rmse_Ld: [1.8253177 2.2408202 2.5627003]


## Run TES

In [15]:
lst_true = np.load('result/tes/lst_true.npy') # true LST on the simulated test set
idx_test = pickle.load(open('data/simulation_dataset/idx.pkl', 'rb'))['test']
Lg_true = np.load('data/simulation_dataset/Lg_simu.npy')[idx_test][..., [1,3,4]]
Ld_true = np.load('data/simulation_dataset/Ld.npy')[idx_test][..., [1,3,4]]
Ld_true = Ld_true[:,None,None,None,:] * np.ones_like(Lg_true)
Lg_sw_tes = np.load('result/sw_tes/Lg_sw_tes.npy')
Ld_sw_tes = np.load('result/sw_tes/Ld_sw_tes.npy')[idx_test]
Ld_sw_tes = Ld_sw_tes[:,None,None,None,:] * np.ones_like(Lg_true)
Lg_true, Ld_true, Ld_sw_tes = Lg_true.reshape(-1, 3), Ld_true.reshape(-1, 3), Ld_sw_tes.reshape(-1, 3)

# reshape the input radiances to meet the requirements of TES codes
Lg_true, Ld_true = Lg_true.swapaxes(0,1)[:,:,np.newaxis], Ld_true.swapaxes(0,1)[:,:,np.newaxis]
Lg_sw_tes, Ld_sw_tes = Lg_sw_tes.swapaxes(0,1)[:,:,np.newaxis], Ld_sw_tes.swapaxes(0,1)[:,:,np.newaxis]

In [ ]:
# if you want to test the role of Lg and Ld, combine Lg_true / Lg_sw_tes and Ld_true / Ld_sw_tes
lst_sw_tes, _, qa_sw_tes, _ = tes_calculator(Lg_sw_tes, Ld_sw_tes)
lst_sw_tes = lst_sw_tes.squeeze()
qa_sw_tes = qa_sw_tes.squeeze()
rmse = root_mean_squared_error(lst_true[qa_sw_tes==1], lst_sw_tes[qa_sw_tes==1])
bias = np.mean(lst_sw_tes[qa_sw_tes==1] - lst_true[qa_sw_tes==1])

# save tes results
np.save('result/sw_tes/lst_sw_tes.npy', lst_sw_tes.astype(np.float32))
np.save('result/sw_tes/qa_sw_tes.npy', qa_sw_tes.astype(np.float32))
savemat('matlab_plot/data/lst_sw_tes.mat', {'lst_true': lst_true[qa_sw_tes==1].astype(np.float32), 'lst_prd': lst_sw_tes[qa_sw_tes==1].astype(np.float32)})

TES starting...
NEM starting...
NEM Iteration: 1
Undecided pixels: 1014450
NEM Iteration: 2
Undecided pixels: 1014450
NEM Iteration: 3
Undecided pixels: 1014258
NEM Iteration: 4
Undecided pixels: 624555
NEM Iteration: 5
Undecided pixels: 398268
NEM Iteration: 6
Undecided pixels: 232166
NEM Iteration: 7
Undecided pixels: 132245
NEM Iteration: 8
Undecided pixels: 80754
NEM Iteration: 9
Undecided pixels: 53616
NEM Iteration: 10
Undecided pixels: 39074
NEM Iteration: 11
Undecided pixels: 30150
NEM Iteration: 12
Undecided pixels: 24398
NEM Iteration: 13
Undecided pixels: 20428
All pixels completed
NEM starting...
NEM Iteration: 1
Undecided pixels: 391199
NEM Iteration: 2
Undecided pixels: 391199
NEM Iteration: 3
Undecided pixels: 391199
NEM Iteration: 4
Undecided pixels: 351281
NEM Iteration: 5
Undecided pixels: 262169
NEM Iteration: 6
Undecided pixels: 160763
NEM Iteration: 7
Undecided pixels: 90140
NEM Iteration: 8
Undecided pixels: 52935
NEM Iteration: 9
Undecided pixels: 32605
NEM Itera

In [ ]:
# calculate RMSE at each tpw and lst bin
tpw = np.load('data/simulation_dataset/tpw.npy')[idx_test]
tpw = tpw[:, None, None, None] * np.ones((len(idx_test), num_vza, num_lst, num_emi))
tpw = tpw.reshape(-1)
lst_edges = np.arange(180, 351, 20)
nums = np.zeros((7, 9)).astype(int)
rmses = np.zeros((7, 9))

for i in range(0, 7):
    for j in range(9):
        if j!=8:
            idx_ij = np.where((tpw >= i*10) & (tpw < i*10+10) & (lst_true >= lst_edges[j]) & (lst_true < lst_edges[j+1]))
        else:
            idx_ij = np.where((tpw >= i*10) & (tpw < i*10+10) & (lst_true >= lst_edges[j]))
        
        nums[i,j] = len(idx_ij[0])
        
        lst_ij, lst_prd_ij = lst_true[idx_ij], lst_sw_tes[idx_ij]
        qa_ij = qa_sw_tes[idx_ij]

        if len(lst_prd_ij[qa_ij==1]) > 0:
            rmses[i,j] = root_mean_squared_error(lst_ij[qa_ij==1], lst_prd_ij[qa_ij==1])

rmses[rmses==0] = np.nan
# savemat('matlab_plot/data/rmse_bins_sw_tes.mat', {'rmses': rmses.astype(np.float32)})

# SWDTES (Wang et al., 2024)  
## Calibrate coefficients

In [33]:
## load simulation dataset
# TOA radiances -> TOA BT
Lt_simu = np.load('data/simulation_dataset/Lt_simu.npy')[...,[1,3,4]]
num_atm, num_vza, num_lst, num_emi, num_channel  = Lt_simu.shape
# convert radiance to brightness temperature
Tt_simu = Inverse_Planck_law(Lt_simu, lamda_c)

# LST and LSE
lst = np.load('data/simulation_dataset/skt.npy')
lst = lst[:,None,:,None] * np.ones_like(Tt_simu[...,0])
emi = np.load('data/simulation_dataset/emi.npy')[..., [1,3,4]]
emi = emi[:,None,...] * np.ones_like(Tt_simu)

# PWV
tpw = np.load('data/simulation_dataset/tpw.npy') # unit: mm
tpw = tpw[:,None,None,None] * np.ones_like(lst)

# split train and test dataset
idx = pickle.load(open('data/simulation_dataset/idx.pkl', 'rb'))
idx_train, idx_val, idx_test = idx['train'], idx['val'], idx['test']

lst_train, lst_test = lst[idx_train].reshape(-1), lst[idx_test].reshape(-1)

In [8]:
# construct input variants
T1_T2_2 = (Tt_simu[...,0] - Tt_simu[...,1])**2
T1_T3_2 = (Tt_simu[...,0] - Tt_simu[...,2])**2
T2_T3_2 = (Tt_simu[...,1] - Tt_simu[...,2])**2
w_lse = tpw[..., None] * emi

# construct input matrix
X_base = np.concatenate([Tt_simu, T1_T2_2[...,None], T1_T3_2[...,None], T2_T3_2[...,None], tpw[...,None]], axis=-1)
X1 = np.concatenate([X_base, emi[..., 0:1], w_lse[...,0:1]], axis=-1)
X2 = np.concatenate([X_base, emi[..., 1:2], w_lse[...,1:2]], axis=-1)
X3 = np.concatenate([X_base, emi[..., 2:3], w_lse[...,2:3]], axis=-1)

X1_train, X1_test = X1[idx_train].reshape(-1, X1.shape[-1]), X1[idx_test].reshape(-1, X1.shape[-1])
X2_train, X2_test = X2[idx_train].reshape(-1, X1.shape[-1]), X2[idx_test].reshape(-1, X1.shape[-1])
X3_train, X3_test = X3[idx_train].reshape(-1, X1.shape[-1]), X3[idx_test].reshape(-1, X1.shape[-1])

In [9]:
## calibrate coefficients of eqs (16)-(18) in Wang et al. (2024)
model1, model2, model3 = LinearRegression(), LinearRegression(), LinearRegression()
model1.fit(X1_train, lst_train)
model2.fit(X2_train, lst_train)
model3.fit(X3_train, lst_train)

# loss on trainset
lst_train_pred1, lst_train_pred2, lst_train_pred3 = model1.predict(X1_train), model2.predict(X2_train), model3.predict(X3_train)
rmse_train1, rmse_train2, rmse_train3 = root_mean_squared_error(lst_train, lst_train_pred1), root_mean_squared_error(lst_train, lst_train_pred2), root_mean_squared_error(lst_train, lst_train_pred3)

# loss on testset
lst_test_pred1, lst_test_pred2, lst_test_pred3 = model1.predict(X1_test), model2.predict(X2_test), model3.predict(X3_test)
rmse_test1, rmse_test2, rmse_test3 = root_mean_squared_error(lst_test, lst_test_pred1), root_mean_squared_error(lst_test, lst_test_pred2), root_mean_squared_error(lst_test, lst_test_pred3)

print('RMSE on trainset: {:.3f}, {:.3f}, {:.3f}'.format(rmse_train1, rmse_train2, rmse_train3))
print('RMSE on testset: {:.3f}, {:.3f}, {:.3f}'.format(rmse_test1, rmse_test2, rmse_test3))

# save coefficients
coefs = np.zeros((3, 10))
coefs[0, 0] = model1.intercept_
coefs[0, 1:] = model1.coef_
coefs[1, 0] = model2.intercept_
coefs[1, 1:] = model2.coef_
coefs[2, 0] = model3.intercept_
coefs[2, 1:] = model3.coef_

# np.save('result/swdtes/coefs_swdtes.npy', coefs)

RMSE on trainset: 1.317, 1.192, 1.991
RMSE on testset: 1.308, 1.176, 1.965


## Apply SWDTES on the simulated test set

In [ ]:
## load simulation dataset
Tt_test = Tt_simu[idx_test].reshape(-1, 3)
tpw_test = tpw[idx_test].reshape(-1)

lst_swdtes, emi_swdtes, qa_swdtes = swdtes_calculator(Tt_test[:,0:1], Tt_test[:,1:2], Tt_test[:,2:3], tpw_test[:,None], t1=0.05)
lst_swdtes, emi_swdtes, qa_swdtes = lst_swdtes.squeeze(), emi_swdtes.squeeze(), qa_swdtes.squeeze()

np.save('result/swdtes/lst_swdtes.npy', lst_swdtes.astype(np.float32))
np.save('result/swdtes/qa_swdtes.npy', qa_swdtes.astype(np.int8))
savemat('matlab_plot/data/lst_swdtes.mat', {'lst_prd': lst_swdtes[qa_swdtes==1].astype(np.float32), 'lst_true': lst_test[qa_swdtes==1].astype(np.float32)})

SWDTES starting...
NEM Iteration: 1
Undecided pixels: 1014450
NEM Iteration: 2
Undecided pixels: 1012893
NEM Iteration: 3
Undecided pixels: 877031
NEM Iteration: 4
Undecided pixels: 538825
NEM Iteration: 5
Undecided pixels: 356281
NEM Iteration: 6
Undecided pixels: 222631
NEM Iteration: 7
Undecided pixels: 151418
NEM Iteration: 8
Undecided pixels: 106555
NEM Iteration: 9
Undecided pixels: 80322
NEM Iteration: 10
Undecided pixels: 62524
NEM Iteration: 11
Undecided pixels: 51291
NEM Iteration: 12
Undecided pixels: 42030
NEM Iteration: 13
Undecided pixels: 35729
All pixels completed
NEM Iteration: 1
Undecided pixels: 284896
NEM Iteration: 2
Undecided pixels: 284774
NEM Iteration: 3
Undecided pixels: 274607
NEM Iteration: 4
Undecided pixels: 176696
NEM Iteration: 5
Undecided pixels: 105690
NEM Iteration: 6
Undecided pixels: 72714
NEM Iteration: 7
Undecided pixels: 48342
NEM Iteration: 8
Undecided pixels: 30674
NEM Iteration: 9
Undecided pixels: 19464
NEM Iteration: 10
Undecided pixels: 120

In [ ]:
# calculate RMSE at each tpw and lst bin
lst_edges = np.arange(180, 351, 20)

nums = np.zeros((7, 9)).astype(int)
rmses = np.zeros((7, 9))

for i in range(0, 7):
    for j in range(9):
        if j!=8:
            idx_ij = np.where((tpw_test >= i*10) & (tpw_test < i*10+10) & (lst_test >= lst_edges[j]) & (lst_test < lst_edges[j+1]))
        else:
            idx_ij = np.where((tpw_test >= i*10) & (tpw_test < i*10+10) & (lst_test >= lst_edges[j]))
        
        nums[i,j] = len(idx_ij[0])
        
        lst_ij, lst_prd_ij = lst_test[idx_ij], lst_swdtes[idx_ij]
        qa_ij = qa_swdtes[idx_ij]

        if len(lst_prd_ij[qa_ij==1]) > 0:
            rmses[i,j] = root_mean_squared_error(lst_ij[qa_ij==1], lst_prd_ij[qa_ij==1])

rmses[rmses==0] = np.nan
savemat('matlab_plot/data/rmse_bins_swdtes.mat', {'rmses': rmses.astype(np.float32)})